In [48]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from hmmlearn import hmm

In [ ]:
os.getcwd()

In [3]:
os.chdir('..')

fp = 'data/senegal_weather_spei_1970_2020_025x025_v3.csv'

spei_gdf = pd.read_csv(fp)

In [4]:
spei_gdf['date_of_obs'] = pd.to_datetime(spei_gdf['year'].astype(str)+'/'+spei_gdf['month'].astype(str)+'/01')
spei_gdf = spei_gdf.set_index(['id','date_of_obs'])


In [6]:
spei_gdf.head(3)

Unnamed: 0  year  month     gpslat     gpslon  \
id date_of_obs                                                  
6  1970-01-01            0  1970      1  16.692072 -16.043186   
   1970-02-01            1  1970      2  16.692072 -16.043186   
   1970-03-01            2  1970      3  16.692072 -16.043186   

                total_precipitation  temperature  relative_humidity  \
id date_of_obs                                                        
6  1970-01-01                   0.0    23.194412           0.333824   
   1970-02-01                   0.0    25.701786           0.325000   
   1970-03-01                   0.0    25.656452           0.401613   

                  humidex  surface_solar_radiation  ...  \
id date_of_obs                                      ...   
6  1970-01-01   22.962059               231.441176  ...   
   1970-02-01   25.835714               239.714286  ...   
   1970-03-01   27.049355               288.419355  ...   

                soil_temperature_level_1  soil_temperature_level_2  \
id date_of_obs                                                       
6  1970-01-01                  24.512353                 25.345588   
   1970-02-01                  27.333571                 27.025714   
   1970-03-01                  28.188387                 28.153226   

                soil_temperature_level_3  soil_temperature_level_4  spei_1mon  \
id date_of_obs                                                                  
6  1970-01-01                  26.505000                 29.255000   0.061824   
   1970-02-01                  26.521071                 28.273929  -0.595051   
   1970-03-01                  28.197419                 28.140645   0.080660   

                spei_3mon  spei_6mon  spei_12mon  spei_24mon  spei_48mon  
id date_of_obs                                                            
6  1970-01-01         NaN        NaN         NaN         NaN         NaN  
   1970-02-01         NaN        NaN         NaN         NaN         NaN  
   1970-03-01   -0.224773        NaN         NaN         NaN         NaN  

[3 rows x 22 columns]

In [41]:
#single_pt_spei_12 = spei_gdf.loc[6,'1990-01-01':'2020-01-01']['spei_12mon'].reset_index(level=['id']).copy()
single_pt_spei_12 = pd.DataFrame()
single_pt_spei_12.loc[:,'spei_12mon'] = spei_gdf.xs((6,slice('1990-01-01','2020-01-01')),level=('id','date_of_obs'))['spei_12mon']
# single_pt_spei_12.reset_index(level=['id'],inplace=True)
# single_pt_spei_12.drop(columns=['id'],inplace=True)
single_pt_spei_12.head(3)

spei_12mon
id date_of_obs            
6  1990-01-01    -0.328500
   1990-02-01    -0.757287
   1990-03-01    -1.364140

In [32]:
np.linspace(-2.5,2.5,11)
print(np.arange(11,0))

[]


In [44]:
#drought_bins = pd.IntervalIndex.from_tuples([(-5,-2),(-1.999,-1.5),(-1.499,-1),(-0.999,0.999),(1, 1.499),(1.5, 1.999),(2, 5)])
#single_pt_spei_12_classes = pd.cut(single_pt_spei_12,bins = np.linspace(-2.5,2.5,11))#, labels=[1,2,3,4,5,6,7,6,9,10,11])
#single_pt_spei_12_classes = pd.cut(single_pt_spei_12,bins = np.linspace(-2.5,2.5,11),labels=False,retbins=True,right=False)

single_pt_spei_12.loc[:,'spei_12_cat'] = pd.cut(single_pt_spei_12.spei_12mon,bins = np.linspace(-2.5,2.5,11),labels=False)
single_pt_spei_12.head(3)

spei_12mon  spei_12_cat
id date_of_obs                         
6  1990-01-01    -0.328500            4
   1990-02-01    -0.757287            3
   1990-03-01    -1.364140            2

In [45]:
X = single_pt_spei_12.spei_12_cat.to_numpy()

In [53]:
X.reshape(-1,1).shape

(361, 1)

In [54]:
trial_model = hmm.GaussianHMM(n_components=11, covariance_type='full', n_iter=10)
trial_model.fit(X.reshape(-1,1))

C:\Anaconda3\envs\omdena-GPSDD\lib\site-packages\hmmlearn\hmm.py:207: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (11). Possibly due to duplicate points in X.
  kmeans.fit(X)


GaussianHMM(covariance_type='full', n_components=11)

In [58]:
trial_model.score(X.reshape(-1,1))

743.4850096523245

In [67]:
trial_model.predict_proba(np.array([4,3,2,1]).reshape(-1,1))

array([[0.00000000e+000, 0.00000000e+000, 3.64452152e-061,
        9.99932707e-001, 3.64452152e-061, 1.32857331e-009,
        3.64452152e-061, 0.00000000e+000, 5.55663113e-160,
        0.00000000e+000, 6.72918512e-005],
       [0.00000000e+000, 9.99999881e-001, 3.11695367e-018,
        0.00000000e+000, 3.11695367e-018, 4.99654137e-016,
        3.11695367e-018, 0.00000000e+000, 3.10139218e-093,
        0.00000000e+000, 1.18618010e-007],
       [0.00000000e+000, 0.00000000e+000, 5.50673760e-010,
        0.00000000e+000, 5.50673760e-010, 6.32292117e-014,
        5.50673760e-010, 9.99999998e-001, 1.34223903e-099,
        0.00000000e+000, 3.65721141e-010],
       [0.00000000e+000, 0.00000000e+000, 3.33331538e-001,
        0.00000000e+000, 3.33331538e-001, 1.45528285e-013,
        3.33331538e-001, 0.00000000e+000, 2.36271819e-077,
        0.00000000e+000, 5.38624164e-006]])